# PS4 Part 2 — Numerically Solving the Ramsey Model

This notebook implements the algorithm described in the template for Section 2 (Numerically Solving the Ramsey Model). It computes the steady state, searches for the saddle-path initial consumption `c0`, simulates the model, and produces the required plots.

**How to run**: Execute all cells top-to-bottom. The results print `c0` and show two figures.

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt

# Parameters from the template
A = 1.0
alpha = 1.0 / 3.0
sigma = 0.5
rho = 0.05
delta = 0.05
n = 0.02

# Discretization
dt = 0.1
N = 1000

# Error tolerance
tol = 0.01

In [ ]:
def f(k):
    return A * (k ** alpha)


def fprime(k):
    return A * alpha * (k ** (alpha - 1.0))


# Steady state from Euler equation: f'(k*) = delta + rho
k_star = (alpha * A / (delta + rho)) ** (1.0 / (1.0 - alpha))
# Steady state consumption from capital accumulation: c* = f(k*) - (delta + n)k*
c_star = f(k_star) - (delta + n) * k_star

# Initial capital
k0 = 0.5 * k_star

# Basic checks (unit-test style)
assert k_star > 0
assert c_star > 0
assert k0 < k_star

k_star, c_star

## Bisection-style search for the saddle-path `c0`

We simulate the model forward with an Euler scheme. If at any point `c^n > c*`, the guess is too high; if `k^n > k*`, the guess is too low. We iterate until the error metric falls below the tolerance.

In [ ]:
def epsilon_metric(k, c, k_star, c_star):
    return math.sqrt(
        (2.0 * (c - c_star) / (c + c_star)) ** 2
        + (2.0 * (k - k_star) / (k + k_star)) ** 2
    )


def simulate_path(c0, k0, k_star, c_star, N, dt):
    k_path = np.zeros(N + 1)
    c_path = np.zeros(N + 1)
    k_path[0] = k0
    c_path[0] = c0

    for t in range(N):
        k_t = k_path[t]
        c_t = c_path[t]

        # Early exit conditions from the template
        if c_t > c_star:
            return "too_high", k_path[: t + 1], c_path[: t + 1]
        if k_t > k_star:
            return "too_low", k_path[: t + 1], c_path[: t + 1]

        # Euler updates
        k_next = dt * (f(k_t) - c_t - (delta + n) * k_t) + k_t
        c_next = dt * (sigma * (fprime(k_t) - delta - rho)) * c_t + c_t

        # Guard against negative values due to discretization
        k_path[t + 1] = max(k_next, 1e-12)
        c_path[t + 1] = max(c_next, 1e-12)

        # Check convergence at each step
        if epsilon_metric(k_path[t + 1], c_path[t + 1], k_star, c_star) < tol:
            return "converged", k_path[: t + 2], c_path[: t + 2]

    return "no_convergence", k_path, c_path


def find_c0_bisection(k0, k_star, c_star, N, dt, tol, max_iter=60):
    c_low = 0.0
    c_high = c_star
    c0 = 0.5 * c_star

    last_result = None
    for _ in range(max_iter):
        result, k_path, c_path = simulate_path(c0, k0, k_star, c_star, N, dt)
        last_result = (result, k_path, c_path)

        if result == "converged":
            return c0, k_path, c_path, result
        if result == "too_high":
            c_high = c0
        elif result == "too_low":
            c_low = c0
        else:
            # Fallback: use final position to guide bisection
            if c_path[-1] < c_star and k_path[-1] < k_star:
                c_high = c0
            else:
                c_low = c0

        c0 = 0.5 * (c_low + c_high)

    # Return the last attempt even if not converged within max_iter
    return c0, last_result[1], last_result[2], last_result[0]

In [ ]:
c0_star, k_path, c_path, status = find_c0_bisection(k0, k_star, c_star, N, dt, tol)

print("Status:", status)
print("c0 (saddle-path):", c0_star)
print("Final k, c:", k_path[-1], c_path[-1])
print("Final epsilon:", epsilon_metric(k_path[-1], c_path[-1], k_star, c_star))

In [ ]:
# Phase diagram: isoclines and the saddle-path simulation
k_grid = np.linspace(1e-6, k_star * 1.6, 500)

# dot{k}=0 curve
c_dk0 = f(k_grid) - (delta + n) * k_grid

# dot{c}=0 curve (vertical line at k_star)

plt.figure(figsize=(7, 5))
plt.plot(k_grid, c_dk0, label=r"$\dot{k}=0$")
plt.axvline(k_star, color="gray", linestyle="--", label=r"$\dot{c}=0$")
plt.plot(k_path, c_path, color="black", linewidth=2, label="Saddle path")

plt.xlabel("k")
plt.ylabel("c")
plt.title("Phase Diagram with Isoclines")
plt.ylim(bottom=0)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Two alternative paths: c0 above and below saddle-path value
c0_high = min(c_star * 0.999, c0_star * 1.05)
c0_low = max(1e-12, c0_star * 0.95)

status_high, k_high, c_high = simulate_path(c0_high, k0, k_star, c_star, N, dt)
status_low, k_low, c_low = simulate_path(c0_low, k0, k_star, c_star, N, dt)

plt.figure(figsize=(7, 5))
plt.plot(k_path, c_path, color="black", linewidth=2, label="Saddle path")
plt.plot(k_high, c_high, color="red", linestyle="--", label=f"c0 high ({status_high})")
plt.plot(k_low, c_low, color="blue", linestyle=":", label=f"c0 low ({status_low})")
plt.axvline(k_star, color="gray", linestyle="--")
plt.plot(k_grid, c_dk0, color="gray", alpha=0.6)

plt.xlabel("k")
plt.ylabel("c")
plt.title("Alternative Paths Around the Saddle Path")
plt.ylim(bottom=0)
plt.legend()
plt.tight_layout()
plt.show()